In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor, BaggingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder, FunctionTransformer, RobustScaler
from catboost import CatBoostRegressor, Pool
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.integration import CatBoostPruningCallback
# from dython.nominal import associations

%matplotlib inline


In [2]:
print(optuna.__version__)

3.0.3


In [3]:
all_data = pd.read_csv("../../own_data/all_with_stores_pop.csv")
all_data.set_index(["dataset", "range_index"], inplace=True)
all_data['in_mall'] = all_data['mall_name'].notna()
all_data['in_chain'] = all_data['chain_name'].notna()
# all_data['stopplace_type'] = all_data['stopplace_type'].fillna("Mangler type")
all_data['mall_name'] = all_data['mall_name'].fillna("None")
# all_data['address'] = all_data['address'].fillna("None")
# all_data['stopplace_type'] = all_data['stopplace_type'].fillna("None")
# all_data['stopplace_type'] = all_data['stopplace_type'].fillna("None")
all_data['as'] = all_data['store_name'].str.contains(r"\b(AS)\b", case=False, regex=True)
all_data['chain_name'] = all_data['chain_name'].fillna("None")
all_data['busstop_id'] = all_data['busstop_id'].map(str)
all_data['lv1'] = all_data['lv1'].map(str)
all_data['lv2'] = all_data['lv2'].map(str)
all_data['lv3'] = all_data['lv3'].map(str)
all_data['lv4'] = all_data['lv4'].map(str)
all_data.drop(columns=[
    'store_name', 
    'stopplace_type', 
    'address', 
    "importance_level", 
    "mall_name", 
    "busstop_id", 
    # 'other_stores_50', 'buss_stops_1000', 'buss_stops_300',
    # "municipality_name", 
    "lv1", 
    "lv2", 
   #  "lv3", 
    "grunnkrets_id", 
   #  'distance_closest_busstop',
    "lat", 
    "lon", 
    "area_km2", 
    # 'other_stores_50', 
    # 'buss_stops_1000', 'couple_children_0_to_5_years', 'couple_children_18_or_above',
    'couple_children_6_to_17_years', 'couple_without_children_x',
    'single_parent_children_0_to_5_years',
    'single_parent_children_18_or_above',
    'single_parent_children_6_to_17_years', 'singles_x', 'singles_y', 'couple_without_children_y', 'couple_with_children',
    'other_households', 
    'single_parent_with_children',
    'couple_children_0_to_5_years', 'couple_children_18_or_above',
    'side_placement',
      #  'num_of_buss_stops_close', 
      #  'district_age_0-14_distribution',
      #  'district_age_15-34_distribution', 'district_age_35-64_distribution',
      #  'district_age_65-90_distribution', 
      'grunnkrets_population',
      #  'municipality_density', 
      #  'district_density',
       'all_households',
        'lv1_population_district_div_count_stores',
       'lv2_population_district_div_count_stores',
       'lv1_population_municipality_div_count_stores',
       'lv2_population_municipality_div_count_stores',
       'in_mall',
      #  'lv3_population_district_div_count_stores',
      #  'lv4_population_district_div_count_stores',
      #  'lv3_population_municipality_div_count_stores',
      #  'lv4_population_municipality_div_count_stores',
    # 'other_stores_1000', 
    # 'other_stores_250',
   #  'municipality_population', 
   #  'district_population', 
   #  'other_stores_100',

    'district_area',
    'municipality_area',
    # 'in_mall',
      #  'in_chain',
    # 'buss_stops_300'
   #  'lv3'

    ], inplace=True)

data_with_label = all_data.loc[["train"]]

data_with_label.set_index('store_id', inplace=True)
data_without_label = all_data.loc[['test']]
data_without_label.set_index('store_id', inplace=True)
data_without_label.drop(columns=["revenue"], inplace=True)

X, y = data_with_label.loc[:, data_with_label.columns != 'revenue'], data_with_label['revenue']

data_train, data_test = train_test_split(data_with_label, test_size=0.2, random_state=42)

X_train, y_train = data_train.loc[:, data_train.columns != 'revenue'], data_train['revenue']
X_test, y_test = data_test.loc[:, data_test.columns != 'revenue'], data_test['revenue']
test_x_lv4 =[]
test_y_lv4 = []
# d = ["1.1.6.2"]
plaace_lv4 = ['1.1.1.0', '1.1.2.0', '1.1.3.0', '1.1.4.0', '1.1.5.0', '1.1.6.1',
       '1.1.6.2', '1.1.6.3', '1.1.6.4', '1.1.7.0', '1.1.9.0', '1.2.1.0',
       '1.2.2.0', '1.2.3.0', '1.2.4.0', '1.3.1.0', '1.3.2.0', '1.4.1.0',
       '1.4.2.0', '2.1.1.0', '2.1.2.0', '2.1.3.0', '2.1.4.0', '2.1.5.0',
       '2.1.6.0', '2.1.7.0', '2.2.1.0', '2.2.2.0', '2.2.4.0', '2.3.1.0',
       '2.3.2.0', '2.3.3.0', '2.3.4.0', '2.3.5.0', '2.4.1.0', '2.4.2.0',
       '2.4.3.0', '2.4.4.0', '2.4.5.0', '2.4.6.0', '2.5.2.0', '2.6.1.0',
       '2.6.2.0', '2.6.3.1', '2.6.3.2', '2.6.3.3', '2.6.4.0', '2.6.5.0',
       '2.6.6.1', '2.6.6.2', '2.7.1.0', '2.7.2.0', '2.7.3.0', '2.7.4.0',
       '2.7.5.0', '2.7.6.0', '2.8.1.0', '2.8.10.0', '2.8.11.2', '2.8.2.0',
       '2.8.3.0', '2.8.4.0', '2.8.5.0', '2.8.6.0', '2.8.7.0', '2.9.1.0',
       '2.9.2.0', '2.9.3.0', '2.9.4.0', '2.9.5.0', '2.9.7.0', '2.9.8.0',
       '2.9.9.0', '3.2.1.0', '3.2.2.0', '3.2.4.0', '3.3.2.0', '3.3.3.0',
       '3.3.4.0', '3.3.5.0', '3.3.6.0', '3.3.7.0', '3.4.2.0', '3.4.3.0',
       '3.6.1.0']
# for lv_4 in plaace_lv4:
#    lv4_data = data_test[data_test["lv4"] == lv_4]
#    test_slit_x_lv4, test_split_y_lv4 = lv4_data.loc[:, lv4_data.columns != 'revenue'], lv4_data['revenue']
#    test_x_lv4.append(test_slit_x_lv4)
#    test_y_lv4.append(test_split_y_lv4)
y_train_scaled = np.log1p(y_train)
y_test_scaled = np.log1p(y_test)
y_scaled = np.log1p(y)


# Comment in this when testing on test dataset to kaggle
# X_train = X
# y_train = y
# y_train_scaled = y_scaled
# X_test = data_without_label

/tmp/ipykernel_626490/2339654456.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_data['as'] = all_data['store_name'].str.contains(r"\b(AS)\b", case=False, regex=True)


In [4]:
# copied from https://habr.com/ru/sandbox/163469/
import math
class RMSLE(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        result = []
        for index in range(len(targets)):
            val = max(approxes[index], 0)
            der1 = math.log1p(targets[index]) - math.log1p(max(0, approxes[index]))
            der2 = -1 / (max(0, approxes[index]) + 1)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result
        
class RMSLE_val(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * ((math.log1p(max(0, approx[i])) - math.log1p(max(0, target[i])))**2)

        return error_sum, weight_sum

In [5]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    y_pred[y_pred < 0] = 0
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))

In [6]:
numeric_features = X.select_dtypes(include=[np.number]).columns
numeric_features = list(numeric_features.to_numpy())

print(len(numeric_features))

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler())]
)

categorical_features = X.select_dtypes(include=[np.object0]).columns
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(fill_value="missing", strategy="constant")),
        ("onehotencoding", OneHotEncoder(handle_unknown="ignore"))
    ]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="passthrough",
)

categorical_features_include_bool = list(categorical_features.to_numpy())
categorical_features_include_bool.extend(list(X.select_dtypes(include=[np.bool8]).columns.to_numpy()))
categorical_transformer_ordinal = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)
preprocessor_ordinal = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer_ordinal, categorical_features_include_bool)
    ],
    remainder="passthrough",
)

# print((categorical_features_include_bool))

categorical_mask = [True] * len(categorical_features_include_bool) + [False] * len(numeric_features)


25


In [19]:
X_train_ft = preprocessor.fit_transform(X_train)
X_test_ft = preprocessor.transform(X_test)
X_train_ft1, X_val_ft, y_train_scaled1, y_val_scaled = train_test_split(X_train_ft, y_train_scaled, test_size=0.2, random_state=42)
def objective(trial: optuna.Trial) -> float:
        param = {
                "loss_function": trial.suggest_categorical("loss_function", ["Huber:delta=0.5", "RMSE"]),
                "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
                "depth": trial.suggest_int("depth", 1, 12),
                "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
                "bootstrap_type": trial.suggest_categorical(
                "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
                ),
                "n_estimators": trial.suggest_int("n_estimators", 200, 2000),
                "used_ram_limit": "3gb",
                "eval_metric": "RMSE",
                "verbose": 0,
        }
        if param["bootstrap_type"] == "Bayesian":
                param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
        elif param["bootstrap_type"] == "Bernoulli":
                param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

        pruning_callback = CatBoostPruningCallback(trial, "RMSE")
        cat = CatBoostRegressor(**param)
        cat.fit(X_train_ft1, y_train_scaled1, 
                eval_set=[(X_val_ft, y_val_scaled)],
                verbose=0,
                early_stopping_rounds=100,
                callbacks=[pruning_callback])
        pruning_callback.check_pruned()
        # early_stopping_rounds=20)
        # s = cross_val_score(cat, X_train_ft, y_train_scaled, cv=8, scoring="neg_root_mean_squared_error", verbose=0)

        preds = cat.predict(X_val_ft)
        accuracy = rmsle(np.expm1(y_val_scaled), np.expm1(preds))

        return accuracy

In [20]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=15), direction="minimize"
)
study.optimize(objective, n_trials=100, timeout=6000)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-10-19 17:06:17,465] A new study created in memory with name: no-name-cb7a5e62-0496-412f-ae91-dd662cb1cf7b
/tmp/ipykernel_626490/3838617249.py:23: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")
[I 2022-10-19 17:06:20,999] Trial 0 finished with value: 0.7782511067493552 and parameters: {'loss_function': 'Huber:delta=0.5', 'colsample_bylevel': 0.09888416990332971, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'n_estimators': 1145}. Best is trial 0 with value: 0.7782511067493552.
[I 2022-10-19 17:06:29,230] Trial 1 finished with value: 0.7573617437559735 and parameters: {'loss_function': 'RMSE', 'colsample_bylevel': 0.017054411898367802, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 288}. Best is trial 1 with value: 0.7573617437559735.
[I 2022-10-19 17:06:34,499] Trial 2 finished with value: 0.

KeyboardInterrupt: 

In [41]:

param = {'loss_function': 'RMSE', 'colsample_bylevel': 0.05980735726255026, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS',"eval_metric": "RMSE","n_estimators": 411 }
cat = CatBoostRegressor(**param)
cat.fit(X_train_ft,
        y_train_scaled,
        # eval_set=[(X_val_ft, y_val_scaled)],
        verbose=2,
        # early_stopping_rounds=20
        )

Learning rate set to 0.120318
0:	learn: 0.9841942	total: 154ms	remaining: 1m 3s
2:	learn: 0.9114678	total: 649ms	remaining: 1m 28s
4:	learn: 0.8674959	total: 1.08s	remaining: 1m 28s
6:	learn: 0.8424336	total: 1.4s	remaining: 1m 20s
8:	learn: 0.8165446	total: 1.81s	remaining: 1m 21s
10:	learn: 0.7965408	total: 2.22s	remaining: 1m 20s
12:	learn: 0.7805577	total: 2.54s	remaining: 1m 17s
14:	learn: 0.7713248	total: 2.88s	remaining: 1m 15s
16:	learn: 0.7591628	total: 3.21s	remaining: 1m 14s
18:	learn: 0.7530541	total: 3.53s	remaining: 1m 12s
20:	learn: 0.7478661	total: 4.05s	remaining: 1m 15s
22:	learn: 0.7415452	total: 4.47s	remaining: 1m 15s
24:	learn: 0.7359057	total: 4.82s	remaining: 1m 14s
26:	learn: 0.7320694	total: 5.13s	remaining: 1m 12s
28:	learn: 0.7279729	total: 5.43s	remaining: 1m 11s
30:	learn: 0.7251464	total: 5.76s	remaining: 1m 10s
32:	learn: 0.7224215	total: 6.21s	remaining: 1m 11s
34:	learn: 0.7204802	total: 6.53s	remaining: 1m 10s
36:	learn: 0.7187652	total: 6.93s	remaini

In [40]:
preds = cat.predict(X_test_ft)
accuracy = rmsle(np.expm1(y_test_scaled), np.expm1(preds))
print(accuracy)

0.7090734450516947
